## Experiment analysis 

Understand how many chunks are getting sent to LLM when we set a ai_threshold & get the rows that have empty init_chunks

In [1]:
import pandas as pd 
import numpy as np
import requests
import os
from dotenv import load_dotenv
import json
import ast
from tqdm import tqdm

In [81]:
# To get the total chunks that are passed in for each experiment
# Replace the path below with your local results path from the src\data\experiments\cckm_3700\guid
qc_config1_baseline_results_path = r'C:\Users\ha6126\AskATT_DataScience_Experiment_Framework\DataScience_Experiment_Framework\src\data\experiments\cckm_3700\v3\ff8a722a-0cdf-4d2a-817e-fbc4c61fe18d\result.csv'
df = pd.read_csv(qc_config1_baseline_results_path)
# # Define a function to count the elements in the 'data' column

df[f'init_chunks'] = df[f'init_chunks'].apply(eval)
exploded_df = df.explode(f'init_chunks')
count_series = exploded_df.groupby('id')[f'init_chunks'].count()
df['chunk_count'] = df['id'].map(count_series)
print(df['chunk_count'].sum())


5807


In [57]:
#To write the results to a new file
qc_config1_baseline_results_path_new = r'C:\Users\ha6126\AskATT_DataScience_Experiment_Framework\DataScience_Experiment_Framework\src\data\experiments\cckm_3700\v3\ffd31caa-5217-40ce-b303-e2af74001d02\resultnew.csv'
df.to_csv(qc_config1_baseline_results_path_new)


In [77]:
#find the rows that have empty init_chunks
# Replace the path below with your local results path from the src\data\experiments\cckm_3700\guid
qc_config1_baseline_results_path = r'C:\Users\ha6126\AskATT_DataScience_Experiment_Framework\DataScience_Experiment_Framework\src\data\experiments\cckm_3700\v3\e0abc086-d072-4d48-846c-f1983cf94a5c\result.csv'
df = pd.read_csv(qc_config1_baseline_results_path)
count_value1_mask = (df['init_chunks'] == '[]').sum()
print(count_value1_mask)

8


In [78]:
# Print the IDs that are sending empty chunks
rows_with_value_empty = df['id'][df['init_chunks'] == '[]'].tolist()
print(rows_with_value_empty)

[10030, 10149, 20122, 20149, 30149, 50029, 50037, 50049]


In [79]:
# Print all columns
empty_rows = df[df['init_chunks'] == '[]']
print(empty_rows)

        id Total Views                                         source  \
18   10030      112089             Customer Contact Roles - BCSS.html   
110  10149       18228            Home Market Information - BCSS.html   
255  20122       24251        Sales Execution Advisor Hub - BCSS.html   
266  20149       18228            Home Market Information - BCSS.html   
360  30149       18228            Home Market Information - BCSS.html   
399  50029    disliked                    International traveling hub   
406  50037    disliked  Processing DIRECTV FOR BUSINESS Orders - BCSS   
418  50049    disliked         Telegence Payment-Related Codes - BCSS   

                                         bcss_question  \
18                                  What can AOP's do?   
110  What billing system or market is North Carolin...   
255                   where are last months avisories?   
266  What billing system or market is New York loca...   
360  What billing system or market is Puerto Rico l.

In [80]:
# Print specific columns
empty_rows_QA = df.loc[df['init_chunks'] == '[]',['id','article_number','bcss_question','answer','generated_answer','gpt-4-32k_score']]
print(empty_rows_QA)

        id  article_number                                      bcss_question  \
18   10030          9260.0                                 What can AOP's do?   
110  10149         26935.0  What billing system or market is North Carolin...   
255  20122         32221.0                   where are last months avisories?   
266  20149         26935.0  What billing system or market is New York loca...   
360  30149         26935.0  What billing system or market is Puerto Rico l...   
399  50029         15823.0  what is calling rate for london on att world c...   
406  50037          5225.0               what's the cheapest directv package?   
418  50049         23743.0                    What if the code is not listed?   

                                                answer  \
18   ï¿½\tCan place orders or make billing account ...   
110                 Telegence\nGAC\nTennessee/Kentucky   
255  Archived advisors can be found by clicking on ...   
266  Telegence\nMNY\nBOS - Boston MA\n

In [137]:
# To get the total chunks that are passed in for each experiment
# Replace the path below with your local results path from the src\data\experiments\cckm_3700\guid
qc_config1_baseline_results_path = r'C:\Users\ha6126\AskATT_DataScience_Experiment_Framework\DataScience_Experiment_Framework\src\data\experiments\cckm_3700\v3\ff8a722a-0cdf-4d2a-817e-fbc4c61fe18d\result.csv'
df = pd.read_csv(qc_config1_baseline_results_path)

In [138]:
#Identify the reranker score of the ground truth article
df[f'init_chunks'] = df[f'init_chunks'].apply(eval)
exploded_data_df = df.explode(f'init_chunks')

exploded_data_df["chunk_source_article_number"] = exploded_data_df.apply(
    lambda x: int(x['init_chunks'][1]) if isinstance(x['init_chunks'], list) or isinstance(x['init_chunks'], tuple) else str(),  # Cast to int because of leading 0s in the returned article number string 
    axis = 1
)

exploded_data_df["chunk_cog_reranker_score"] = exploded_data_df.apply(
    lambda x: x['init_chunks'][2] if isinstance(x['init_chunks'], list) or isinstance(x['init_chunks'], tuple) else str(),
    axis = 1
)

exploded_data_df['match'] = exploded_data_df['chunk_cog_reranker_score'].where(
    exploded_data_df['article_number'] == exploded_data_df['chunk_source_article_number'], 0)   

#Identify the article id with the best reranker score
exploded_data_df['best_article'] = exploded_data_df['chunk_source_article_number'].where(
    exploded_data_df['chunk_cog_reranker_score'] == exploded_data_df['max_cog_reranker_score'], 0)  
                  
exploded_data_df[['id','bcss_question','match','chunk_cog_reranker_score','max_cog_reranker_score']].head(5)

# Compute the max cog recall score
df = df.merge(
    exploded_data_df.groupby("id")[f"match"].max(),
    on = "id",
    how = "left"
)

df = df.merge(
    exploded_data_df.groupby("id")[f"best_article"].max(),
    on = "id",
    how = "left"
)
df['is_match'] = np.where(df['article_number'] == df['best_article'], True, False)


df[['id','bcss_question','article_number','match','in_top_init','best_article','max_cog_reranker_score','is_match']].head(5)
# exploded_data_df['best_article'] = exploded_data_df['best_article'].astype(int)
# exploded_data_df['article_number'] = exploded_data_df['article_number'].astype(int)

,id,bcss_question,article_number,match,in_top_init,best_article,max_cog_reranker_score,is_match
0,10000,What international benefits are included in th...,21757.0,3.130668,1,30146,3.395680,False
1,10003,Can I call the AOP/TCM if the verfied end-user...,16479.0,2.574602,1,16817,2.761872,False
2,10005,How many digits can a PIN be?,7152.0,3.286152,3,7152,3.286152,True
3,10006,What is a mobile hotspot?,7260.0,3.944853,5,7260,3.944853,True
4,10008,What countries are eligible for the Internatio...,4733.0,0.000000,0,33664,3.033395,False


In [135]:
df['is_top'] = df['in_top_init'] > 0
correlation = df['is_top'].corr(df['is_match'])
print(f"The correlation is {correlation}")

The correlation is 0.3062600473685392


In [136]:
correlation = df['max_cog_reranker_score'].corr(df['is_match'])
print(f"The correlation is {correlation}")

The correlation is 0.10844797954358043


In [140]:
#write out the results to a new file
qc_config1_baseline_results_path_new = r'C:\Users\ha6126\AskATT_DataScience_Experiment_Framework\DataScience_Experiment_Framework\src\data\experiments\cckm_3700\v3\ff8a722a-0cdf-4d2a-817e-fbc4c61fe18d\resultnew.csv'
df.to_csv(qc_config1_baseline_results_path_new)